In [1]:
import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import re
import os
from sklearn.preprocessing import MinMaxScaler

In [2]:
books = pd.read_excel('C://Users/DS7_RVepuri/Desktop/Reccomendation/User 4.xlsx',sheet_name='user_item4')
items = pd.read_excel('C://Users/DS7_RVepuri/Desktop/Reccomendation/User 4.xlsx',sheet_name='Item-desc')
data = pd.merge(books,items)

In [3]:
books.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
def clean_title(statement):
    return ' '.join(re.sub("/@%&|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", statement).split())

In [5]:
for i in items['title'].index:
    items['title'][i] = clean_title(items['title'][i])

In [6]:
data = pd.merge(books,items,on='ITEM')

In [7]:
data.head()

,USER,ITEM,RATING,title
0,A2NVNMLC0AFL08,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones
1,A2UZUH4QHV4HA1,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones
2,A2IX2PCXS5O2FK,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones
3,AWG2Y42W9RNNK,B004KNWWMW,4,Fostex T40RP MkII Pro Headphones
4,A1YGUFU306JKGK,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones


In [8]:
# Create a numeric user_id and title_id column
data['USER'] = data['USER'].astype("category")
data['title'] = data['title'].astype("category")
data['user_id'] = data['USER'].cat.codes
data['title_id'] = data['title'].cat.codes

In [9]:
# Create a lookup frame so we can get the title names back in  readable form later.
item_lookup = data[['title_id', 'title']].drop_duplicates()
item_lookup['title_id'] = item_lookup.title_id.astype(str)

user_lookup = data[['user_id', 'USER']].drop_duplicates()
user_lookup['user_id'] = item_lookup.title_id.astype(str)

In [10]:
data.head()

,USER,ITEM,RATING,title,user_id,title_id
0,A2NVNMLC0AFL08,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones,2174,23
1,A2UZUH4QHV4HA1,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones,2465,23
2,A2IX2PCXS5O2FK,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones,1982,23
3,AWG2Y42W9RNNK,B004KNWWMW,4,Fostex T40RP MkII Pro Headphones,5082,23
4,A1YGUFU306JKGK,B004KNWWMW,5,Fostex T40RP MkII Pro Headphones,1136,23


In [11]:
data = data.drop(['USER', 'ITEM','title'], axis=1)


In [12]:
#data = data.drop(['mcat_id'], axis = 1)

In [13]:
data = data.loc[data.RATING != 0]


In [14]:
# Create lists of all users, items and their purchase counts
users = list(np.sort(data.user_id.unique()))
title = list(np.sort(data.title_id.unique()))
RATINGS = list(data.RATING)

In [15]:
# Get the rows and columns for our new matrix
rows = data.user_id.astype(int)
cols = data.title_id.astype(int)

In [16]:
 # Create a sparse matrix for our users and items containing number of Ratings
data_sparse_new = sparse.csr_matrix((RATINGS, (rows, cols)), shape=(len(users), len(title)))

In [17]:
print(data_sparse_new)

  (0, 3)	4
  (1, 27)	5
  (2, 83)	4
  (3, 51)	5
  (4, 34)	15
  (5, 90)	2
  (6, 31)	2
  (6, 74)	3
  (7, 8)	4
  (8, 61)	3
  (9, 29)	4
  (9, 98)	4
  (10, 0)	4
  (10, 35)	2
  (10, 59)	1
  (11, 79)	6
  (12, 76)	8
  (13, 70)	4
  (14, 101)	3
  (15, 48)	4
  (15, 52)	4
  (15, 60)	4
  (15, 69)	3
  (16, 56)	4
  (17, 2)	4
  :	:
  (5195, 92)	3
  (5196, 62)	4
  (5197, 38)	1
  (5198, 31)	1
  (5199, 81)	3
  (5200, 21)	3
  (5200, 31)	1
  (5200, 37)	3
  (5200, 40)	3
  (5201, 43)	3
  (5202, 19)	3
  (5203, 46)	3
  (5204, 0)	3
  (5204, 96)	3
  (5205, 18)	3
  (5206, 11)	3
  (5207, 76)	8
  (5208, 18)	2
  (5209, 62)	3
  (5210, 46)	1
  (5210, 60)	3
  (5211, 48)	4
  (5212, 30)	2
  (5212, 84)	3
  (5213, 87)	1


In [18]:

def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
     # Calculate the Confidence for each value in our data
    confidence = sparse_data * alpha_val
    
    # Get the size of user rows and item columns using numpy array shape
    user_size, item_size = sparse_data.shape
    
    # We create the user vectors X of size users x features, the item vectors
    # Y of size items x features and randomly assign values to them using np.random.normal
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))
    
    #Identity matrix and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)
    
    I = sparse.eye(features)
    lI = lambda_val * I
    
    for i in range(iterations):
        print('iteration %d of %d' % (i+1, iterations))
        
        # Precompute Y-transpose-Y and X-transpose-X
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Run in a loop for entire user data
        for u in range(user_size):

            # Get the user row.
            u_row = confidence[u,:].toarray() 

            # Calculate the binary preference p(u)
            p_u = u_row.copy()
            p_u[p_u != 0] = 1.0

            # Calculate Cu and Cu - I
            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            # Put it all together and compute the final formula
            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)

    
        for i in range(item_size):

            # Get the item column and transpose it.
            i_row = confidence[:,i].T.toarray()

            # Calculate the binary preference p(i)
            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            # Calculate Ci and Ci - I
            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            # Put it all together and compute the final formula
            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y


In [22]:

user_vecs1, item_vecs1 = implicit_als(data_sparse_new, iterations=5, features=20, alpha_val=40)
# Let's say we want to recommend mcats for user having user ID = 11
USER = 11

#------------------------------
# GET the buy leads purchased by the user
#------------------------------

# Let's print out what the user has purchased
consumed_idx = data_sparse_new[USER,:].nonzero()[1].astype(str)
consumed_items = item_lookup.loc[item_lookup.title_id.isin(consumed_idx)]
print(consumed_items)


#------------------------------
# Lets create users recommendations
#------------------------------

def recommend(USER, data_sparse_new, user_vecs1, item_vecs1, item_lookup, num_items=10):

    # Get all interactions by the user
    user_interactions = data_sparse_new[USER,:].toarray()

    # We don't want to recommend items the user has consumed. So let's
    # set them all to 0 and the unknowns to 1.
    user_interactions = user_interactions.reshape(-1) + 1 #Reshape to turn into 1D array
    user_interactions[user_interactions > 1] = 0

    # This is where we calculate the recommendation by taking the 
    # dot-product of the user vectors with the item vectors.
    rec_vector = user_vecs1[USER,:].dot(item_vecs1.T).toarray()

    # Let's scale our scores between 0 and 1 to make it all easier to interpret.
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions*rec_vector_scaled
   
    # Get all the mcats indices in order of recommendations (descending) and
    # select only the top mcats. 
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    Items_recommended = []
    scores = []

    # Loop through our recommended mcat indicies and look up the actual mcat name
    for idx in item_idx:
        Items_recommended.append(item_lookup.title.loc[item_lookup.title_id == str(idx)].iloc[0])
        scores.append(recommend_vector[idx])

    # Create a new dataframe with recommended mcat names and scores
    recommendations = pd.DataFrame({'Items_recommended': Items_recommended, 'score': scores})
    
    return recommendations

# Let's generate and print our recommendations of mcats against users
recommendations = recommend(USER, data_sparse_new, user_vecs1, item_vecs1, item_lookup)
print(recommendations)


iteration 1 of 5
iteration 2 of 5
iteration 3 of 5
iteration 4 of 5
iteration 5 of 5
     title_id                                              title
6455       79  Skagen Men s 596XLTXM Multifunction Titanium B...
                                   Items_recommended     score
0                                             Xender  0.553025
1  Safe At Office 500 Wrls 5U Internet Security A...  0.546691
2                      DVI Gear HDMI Cable 2M 6 feet  0.543867
3  OtterBox Defender Series Case for iPhone 5 Dis...  0.537740
4  Comprehensive ST Series USB Cable 2 0 A to B C...  0.537683
5                                Hijacked Flight 285  0.523712
6   KEEN Men s Targhee II Mid Waterproof Hiking Boot  0.521595
7               Computer Or Monitor Power Cable 50FT  0.515640
8  Kroo Forza Leather Case for Creative Zen Visio...  0.515640
9  Rectangular Dark Orange Enamel Silver Cufflink...  0.509827
